In [1]:
import pickle
import random
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
import os
import matplotlib.pyplot as plt
import random
from PIL import Image
import  cv2
from imblearn.over_sampling import SMOTE
from keras.preprocessing.image import ImageDataGenerator

# Oversample with SMOTE and random undersample for imbalanced dataset
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from torchvision import transforms

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
def numimg(dirpath):
  # Get a list of all files in the directory
  files = os.listdir(dirpath)
  print(len(files))

In [4]:
print(numimg('/content/drive/MyDrive/TubeXRAY/Normal'),numimg('/content/drive/MyDrive/TubeXRAY/Tuberculosis'))

3500
3500
None None


In [5]:
def load_images_from_folder(folder, images):
    o = 0
    for filename in os.listdir(folder):
        if filename.endswith(".jpg"):
            o += 1
            img = cv2.imread(os.path.join(folder, filename))
            img = cv2.resize(img, (256, 256))
            images.append(img)
    print(o)

In [6]:
imgs=[]
filebasenormal='/content/drive/MyDrive/TubeXRAY/Normal'
filebasetuber='/content/drive/MyDrive/TubeXRAY/Tuberculosis'

load_images_from_folder(filebasenormal,imgs)
load_images_from_folder(filebasetuber,imgs)


3499
3500


In [7]:
import numpy as np


zeros_arr = np.zeros(3499,dtype=int)
ones_arr = np.ones(3500,dtype=int)
labels= np.concatenate((zeros_arr, ones_arr))

imgs=np.array(imgs)
print(labels)

print(imgs.shape)

[0 0 0 ... 1 1 1]
(6999, 256, 256, 3)


In [8]:
# combine images and labels random

indices = np.arange(len(labels))
np.random.shuffle(indices)
imgs = imgs[indices]
labels = labels[indices]

In [9]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(imgs, labels, test_size=0.2, random_state=42)

In [10]:
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(5599, 256, 256, 3) (1400, 256, 256, 3) (5599,) (1400,)


In [15]:
from tensorflow.keras.applications.resnet50 import ResNet50
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras.optimizers import Adam
import keras



base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(2, activation='softmax')(x)


model = Model(inputs=base_model.input, outputs=predictions)


for layer in base_model.layers:
    layer.trainable = False


model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [12]:
from keras.utils import to_categorical
# تبدیل برچسب‌ها به شکل one-hot encoding
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

print(y_train)

[[0. 1.]
 [0. 1.]
 [1. 0.]
 ...
 [0. 1.]
 [1. 0.]
 [1. 0.]]


In [16]:
# آموزش مدل
history=model.fit(x_train, y_train, validation_split=0.15, epochs=10, batch_size=32)

Epoch 1/10
149/149 [==============================] - 28s 161ms/step - loss: 0.2939 - accuracy: 0.9281 - val_loss: 0.0619 - val_accuracy: 0.9786
Epoch 2/10
149/149 [==============================] - 23s 155ms/step - loss: 0.0585 - accuracy: 0.9779 - val_loss: 0.0402 - val_accuracy: 0.9857
Epoch 3/10
149/149 [==============================] - 23s 153ms/step - loss: 0.0383 - accuracy: 0.9861 - val_loss: 0.1108 - val_accuracy: 0.9560
Epoch 4/10
149/149 [==============================] - 23s 153ms/step - loss: 0.0214 - accuracy: 0.9922 - val_loss: 0.0312 - val_accuracy: 0.9893
Epoch 5/10
149/149 [==============================] - 21s 140ms/step - loss: 0.0129 - accuracy: 0.9954 - val_loss: 0.0392 - val_accuracy: 0.9821
Epoch 6/10
149/149 [==============================] - 21s 140ms/step - loss: 0.0087 - accuracy: 0.9964 - val_loss: 0.0167 - val_accuracy: 0.9929
Epoch 7/10
149/149 [==============================] - 21s 138ms/step - loss: 0.0047 - accuracy: 0.9987 - val_loss: 0.0255 - val_ac

In [17]:
test_loss, test_accuracy = model.evaluate(x_test, y_test)
print(test_loss,test_accuracy)

44/44 [==============================] - 5s 117ms/step - loss: 0.0059 - accuracy: 0.9986
0.005911742802709341 0.9985714554786682
